<a href="https://colab.research.google.com/github/Harshini-2k5/dl/blob/main/vgg16exp7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
dataset_name = "cats_vs_dogs"
dataset, info = tfds.load(dataset_name, as_supervised=True, with_info=True)
train_data = dataset['train'].take(5000)
val_data = dataset['train'].skip(5000).take(1000)

In [2]:
def preprocess(image, label):
    image = tf.image.resize_with_pad(image, 224, 224)
    image = image / 255.0
    return image, label
train_data = train_data.map(preprocess).batch(128).shuffle(1000)
val_data = val_data.map(preprocess).batch(128)
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3),include_top=False, weights='imagenet')
base_model.trainable = False
model = tf.keras.Sequential([base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, validation_data=val_data, epochs=3)
loss, acc = model.evaluate(val_data)
print(f"\nValidation Accuracy: {acc * 100:.2f}%")
def show_prediction():
    image, label = next(iter(val_data))
    img = image[0].numpy()
    true_label = label[0].numpy()

    prediction = model.predict(tf.expand_dims(image[0], axis=0))
    predicted_label = "Dog" if prediction[0][0] > 0.5 else "Cat"
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_label}, Actual: {'Dog' if true_label else 'Cat'}")
    plt.axis("off")
    plt.show()

show_prediction()

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 2491s 62s/step - accuracy: 0.6930 - loss: 1.4113 - val_accuracy: 0.9040 - val_loss: 0.2326
Epoch 2/3
35/40 ━━━━━━━━━━━━━━━━━━━━ 4:15 51s/step - accuracy: 0.9010 - loss: 0.2339